# Example Multiprocessing Merge

This example consists of three files of a simple example of multiprocessing. The files are:
<ol>
    <li>
        example_multiprocessing_merge.py
    </li>
    <li>
        example_multiprocessing_merge_root.py
    </li>
    <li>
        example_multiprocessing_merge_leaf.py
    </li>
</ol>

##  Processes and Streams in the Example

The example has a single <i>root</i> process that is fed by streams from an 
arbitrary number of <i>leaf</i> processes.
Each leaf process appends random numbers to an external stream, and the root
process sums the <i>n</i>-th number that it receives on each stream.

This code has two leaf processes called <b>process_x</b> and <b>process_y</b>.
Each leaf process has an external stream called <b>stream_to_root</b> that is
read by the root process.

The target functions of the root and leaf processes are in the files
<i>example_multiprocessing_merge_root.py, example_multiprocessing_merge_leaf.py</i>
respectively. The main program which creates and runs processes is in the file
<i>example_multiprocessing_merge.py</i>.



## Structure of the Code
<ol>
    <li>
        <b><i>Create a dict or other persistent structure to store the input queues
            of processes that are fed by external streams.</i></b> In this example,
            only the <i>root</i> process is fed by external streams. There is only
            one input queue that has to be created.
            <code>
                q_root = mp.Queue()
                dict_queues = {'q_root': q_root}
            </code>
    </li>
    <li>
        <b><i>Create processes.</i></b> Example: Create process_x by specifying (1) the
        target function and (2) the list of arguments to be passed to the target function,
        and optionally (3) a name.
        <code>
            stream_name_x, random_start_x, random_end_x = 'x', 0, 10
            process_x = mp.Process(
               target=example_multiprocessing_merge_leaf.process_target_leaf,
               args=(num_messages, dict_queues, stream_name_x, random_start_x, random_end_x,),
               name='process_x')
        </code>
    </li>
    <li>
        <b><i>Start, join, and terminate processes</i></b>
    </li>
</ol>


In [1]:
import multiprocessing as mp

import example_multiprocessing_merge_leaf
import example_multiprocessing_merge_root

if __name__ == '__main__':
    #----------------------------------------------------------- 
    # 1. CREATE DICT OF INPUT QUEUES OF PROCESSES FED BY EXTERNAL
    # STREAMS.
    #----------------------------------------------------------- 
    q_root = mp.Queue()
    dict_queues = {'q_root': q_root}
    
    #----------------------------------------------------------- 
    # 2. CREATE PROCESSES
    #----------------------------------------------------------- 
    num_messages = 4

    # PROCESS_X
    stream_name_x, random_start_x, random_end_x = 'x', 0, 10
    process_x = mp.Process(
        target=example_multiprocessing_merge_leaf.process_target_leaf,
        args=(num_messages, dict_queues, stream_name_x, random_start_x, random_end_x,),
        name='process_x')

    # PROCESS_Y
    stream_name_y, random_start_y, random_end_y = 'y', 100, 110
    process_y = mp.Process(
        target=\
        example_multiprocessing_merge_leaf.process_target_leaf,
        args=(num_messages, dict_queues, stream_name_y, random_start_y, random_end_y,), 
        name='process_y')

    # PROCESS_MERGE
    process_merge = mp.Process(
        target=\
        example_multiprocessing_merge_root.process_target_merge,
        args=(dict_queues,),
        name='process_merge')
    
    #----------------------------------------------------------- 
    # 3. START, JOIN, TERMINATE PROCESSES
    #----------------------------------------------------------- 
    process_x.daemon = True
    process_y.daemon = True
    process_merge.daemon = True

    process_x.start()
    process_y.start()
    process_merge.start()

    process_x.join()
    process_y.join()
    process_merge.join()
    
    process_x.terminate()
    process_y.terminate()
    process_merge.terminate()

x 3
x 5
x 3
x 9
y 108
y 106
y 106
y 109
merge  111
merge  111
merge  109
merge  118
